## Abstract

This research merged 5 years of flight and weather data from 2015-2019 to predict flight departure delays exceeding 15 minutes up to two hours before departure. Using data from 2015-2018 for training and 2019 data for validation, cross-validation with grid search optimized model hyperparameters. Key engineered features included prior flight delays exceeding two hours based on airplane tail number, days away from federal holidays, and the number of flights from the origin airport. Despite introducing deep learning models, ensemble methods like Random Forests and Gradient-Boosted Trees alongside Logistic Regression were prioritized. All models showed similar Area Under the ROC Curve (AUC) results, with Gradient-Boosted Trees yielding an AUC of 0.788, proving slightly superior in terms of performance and time efficiency. The model boasts a specificity above 95%, predicting over 38% of flight delays two hours in advance, highlighting its practical significance. Future enhancements include deeper feature engineering and leveraging more computational resources for advanced modeling.

## Discussion of Data:

##### Source and Composition:
The dataset is a combination of flight information and weather metrics spanning five consecutive years, from 2015 to 2019. All the weather data comes from NOAA, and all of the flight information comes directly from the department of transportation.

##### Temporal Distribution:
The temporal span ensures capturing variations that occur cyclically, including seasonal weather changes, annual flight trends, peak travel times, and holiday-induced fluctuations. With this duration, we can factor in both short-term anomalies and long-term patterns.

##### Granularity and Volume:
Given that the data contains five years of flight and weather recordings, the dataset is rich in entries, totaling around 32 million observations in the 5 year span. This extensive volume allows for the possibility of robust model building, though it does come with challenges concerning data processing and model training, especially given the time intensity of training on the complete dataset.

##### Data Quality:
One recurring theme in this data journey is the trade-off between quality and quantity. While vast amounts of data often provide richer insights, it's crucial that the data is clean, consistent, and devoid of too many null values. The decision to omit features with a majority of null values showcases a commitment to data integrity, even if it sacrifices some potentially useful information.

##### Data Leakage:
To circumvent data leakage, we instituted a stringent rule while engineering features: a window of 2 hours before the flight's departure was strictly maintained to ensure no leakage of the target variable. This boundary acted as a safeguard, ensuring that the model was deprived of any premature insights into whether a flight would be delayed or not.

For instance, when assessing factors like previous flight delays based on Airplane Tail Number, this two-hour buffer made sure the model did not inadvertently access data that would directly reveal the delay status of the current flight. By adhering to this temporal boundary, we ensured our features were predictors, not spoilers.

Additionally, we only considered prediction results from our validation set in 2019, which is held out of the model completely. Even though or cross-validation approach technically treated the training samples as I.I.D, we only used it to see which hyperparameters were the most suitable due to time constraints, and then retrained a model on the entire training data before testing on the held out validation data.

## Executive Summary

##### Origin of Dataset

Our dataset was created by joining 5 years of flight data with 5 years of weather data, specifically from the years 2015-2019, coming from the NOAA and DoT.

##### Goal

In this project, we are trying to predict departure delays of greater than 15 minutes up to two hours before departure. 

##### Data/Process

In this phase, we are using 5 years of flight data ranging from 2015-2019. As per instructions, we treat the data from 2015-2018 as training data, and reserve the data from 2019 as a holdout validation set. We perform cross-validation on the training data using a grid search in order to choose optimal hyperparameters before using the trained model on the validation set to judge performance. Additionally, we consider subsets of the training and validation sets in order to speed up the process of evaluating performance, since running on the full dataset takes several hours to complete.

##### Feature Engineering

Our feature engineering was focused on implementing three main features which we believed would lead to the biggest increase in performance. This was based upon both qualitative logic as well as EDA. 

These three features were: 

- (1) Previous flight delays > 2 hours based on Airplane Tail Number
- (2) The absolute minimum days away from a federal holiday
- (3) The number of flights leaving from the origin airport on the flight date.

For feature (1) we believed this would be our strongest feature, as typically a major cause for flight delays is that the plane assigned to the flight arrives later than expected. If we can see that the plane assigned to the flight we are trying to predict has had either a departure delay or an arrival delay before the 2 hour window before the flight, we can use this information to strengthen our classification. The way that we implemented this was using Window functions, and the two hours before scheduled departure columns to cross reference the tail numbers on the same date that have experience departure or arrival days, and then created a binary variable that classified each flight as having a previous route delay or not.

For feature (2) we know that there is a lot more flight traffic during the holidays. However, simply encoding when the holiday is will not be enough, as most people do not fly on the holiday but the days preceeding and following it. That is why we encoded a variable to tell us the date's proximity to a holiday where there might be increased flight traffic and delays. This was an easy feature to implement, as all we had to do was encode the dates of all holidays from 2015 to 2019 and then report the minimum absolute value of the date substracted by the federal holidays. This implementation was not the most efficient way to achieve this, but the runtime was not very bad, so it was fine for this purpose.

Lastly, for feature (3), we noticed in our EDA that certain airports were more prone to delays, especially those that had more flights. This could be due to reasons such as having a harder time assigning gates to departing flights due to a heavier volume. This feature was able to be created by running a group by on flight date and origin airport, followed by reporting the count. We then re-joined this groupby object to the original dataset with a unique identifier.

On top of these three engineered features, we used several features directly from the dataset which we confirmed to both have some relevant connection to delays through EDA, as well as less than a majority of null values. We wanted to make sure we were not adding unpredictive features or biasing the data through imputation.

The feature engineering made a significant difference for us, as it raised our validation AUC from 0.59 in phase 2 to 0.75 in phase 3.

##### Models/Pipeline

Our pipeline takes two sets of columns: numerical and categorical, and provides transforms to both of them involving One-Hot encoding and imputation. After that, they are joined together into one feature space and prepared through vector assembly for the Machine Learning pipeline.

Afterwards, it is put into three separate ML pipelines, two of which are ensemble methods. These are Random Forests and Gradient-Boosted Trees. The third model is a standard Logistic Regression We apply a grid search coupled with Cross-Validation in order to set hyperparameters before applying the model to the validation set.

Additionally, we implemented two deep learning methods: a multi-layer perceptron and an LSTM. However, due to computational limitations, we were not able to find the same success that we did with our classical machine learning approach.

Our evaluator reports the Area Under the ROC Curve as a final evaluation metric.

##### Hyperparameter Tuning

In our machine learning journey, hyperparameter tuning emerged as a crucial step in refining our models. We utilized cross-validation techniques to  explore a range of parameter values, ensuring the model's robustness and reducing the likelihood of overfitting. After identifying the optimal set of hyperparameters, we retrained our model exclusively on the training data. Subsequent testing on a strictly held-out validation set showcased a notable improvement in model performance, with an AUC boost of approximately 0.03, bringing our best model from .76 previous to .788.

##### Findings/Results

All three classifiers showed close performance metrics when measured on AUC.

We considered Logistic Regression to be our baseline estimator given its simplicity in implementation. It yielded an AUC of 0.7735 on the validation data and only took 40 minutes to train.

The Random Forest required approximately 3 hours for training and achieved a validation AUC of 0.762. Meanwhile, the Gradient-Boosted Trees (GBT) method demanded a little over 3 hours for training, returning a validation AUC of 0.788. The GBT model displayed a marginally superior performance compared to Logistic Regression, with Random Forest performing the worst. However, when balancing both time and performance, the Logistic Regression model emerged as the most efficient choice, while GBT was the best-performing one. Given that the GBT model did have a significant enough increase in performance, we would recommend using it, but under time constraints, the LR is a perfectly fine alternative.

##### Conclusions

Conclusively, we succeeded in constructing a robust model with a tangible real-world application. With a specificity exceeding 95%, our model can precisely predict over 38% of plane delays at least 2 hours before their scheduled departure times.

While there remains room for enhancement, such as delving into more intricate feature engineering or allocating more computational resources for a sophisticated deep learning model, we met our foundational goal of creating an effective classifier for detecting flight delays.

With more time, we would likely dedicate time toward looking at the proportion of delays by airport with the same window functions, as well as implementing a the Deep Learning models with GPU and a Voting Classifier with our ML pipelines.

## Metrics: Why we chose AUC + Confusion Matrix

We elected to use the Area Under the Receiver Operating Characteristic Curve (AUC) as our primary evaluation metric because it effectively summarizes a model's ability to distinguish between the positive and negative classes across all threshold values. This choice was particularly pertinent given our project's emphasis on achieving high sensitivity without compromising specificity. In aviation scenarios, the cost of false alarms can be high, but missing a true delay can be even more critical. Thus, maintaining high specificity while achieving reasonable sensitivity was paramount. To further contextualize these trade-offs and provide a holistic view of model performance, we also employed a confusion matrix, offering a granular breakdown of true positives, true negatives, false positives, and false negatives.

## Feature Engineering EDA

##### Previous Flight Delays

This was by far our strongest predictive feature, which is evidenced by the EDA below. We see that departures with a previous delay were nearly 50% likely to be delayed themselves, while those that did not have a previous delay were closer to 12%. This alone counted for much of our improved predictions.

![class_imbalance_depdel15](/files/tables/class_imbalance_depdel15.png)

##### Holiday Distance

This feature was slightly useful, but not amazing, as evidenced by our EDA as well. When looking at flights in the holiday season, there is a slight increase to the chances of delay (18.2% vs 17.8%, but it was nonetheless useful so we decided to include it in our model)

![mean_dep_del15_by_holiday](/files/tables/mean_dep_del15_by_holiday.png)

##### Airport Outlinks

We see that this is also a marginally useful feature, as delayed flights are more likely to come from airports that have more flights leaving that day.

![dist_of_outlinks_by_class_flight_delays](/files/tables/dist_of_outlinks_by_class_flight_delays.png)

##### Hour of the Day

Since we see that the number of departing flights can be a useful feature, we also examined whether the hour of the day impacts the number of flights as well, and we can see that certain hours have many more flights departing.

![eda1](/files/tables/eda1.png)

##### Airline

Certain airlines have a penchant for being more likely to have a delay, we see that this is true in this case, with a wide array of expected outcomes dependent on the airline.

![avg_arrdel_by_airline](/files/tables/avg_arrdel_by_airline.png) 

##### Weather

Bad weather effects are more likely to result in delays in this case, which is to be expected.

![corr_matr](/files/tables/corr_matr.png)

## Cross Validation + Train/Test Approach

In our modeling process, we employed cross-validation on the training dataset, treating the data as if it were Independent and Identically Distributed (I.I.D). This assumption allowed us to divide the training dataset into multiple smaller sets or "folds." By systematically rotating through these folds, using some for training and one for validation in each iteration, we were able to assess the model's stability and performance across different subsets of the data. This iterative evaluation was pivotal for parameter selection, helping us identify optimal hyperparameters that would generalize well. After determining the best parameters using cross-validation, we then retrained our model on the entire training dataset to leverage all available data and improve the model's predictive capacity. Finally, to get a true measure of our model's performance on unseen data, we tested it on the 2019 validation set, which had been completely held out from the initial dataset and parameter tuning process, ensuring an unbiased evaluation.

In our pursuit of creating the most robust predictive model, we not only applied traditional cross-validation but also experimented with a time-series cross-validation approach, specifically tailored for our LSTM deep learning model. This method was particularly suitable for LSTM as it respects the chronological order of the data. In this approach, the training set gradually expanded over time, allowing the validation set to always come after the training data chronologically, ensuring that no future information would unintentionally be used in past predictions. Despite our rigorous efforts with the LSTM, we confronted numerous challenges in its implementation. These challenges, compounded by the computational demands and intricacies of deep learning models, led us to make the difficult decision to sunset our deep learning endeavors in favor of more traditional machine learning models which offered more consistent and interpretable results.

## Machine Learning Algorithms
![all_models](/files/tables/all_models.png)

##### Logistic Regression 

A supervised classification algorithm suitable for predicting binary outcomes. By fitting data to a logistic curve, it estimates the probability that a given instance belongs to a particular category. The model assigns weights to each feature, which signifies their importance in the prediction.

##### Random Forests 
An ensemble learning method that constructs a 'forest' of decision trees during training. Each tree is built on a subset of the data and makes its own predictions. Random Forest then aggregates these individual predictions to produce a final result, ensuring robustness and reducing the likelihood of overfitting. The strength of the Random Forests lies in its ability to handle large data sets with higher dimensionality and its ability to estimate missing values.

##### Gradient-Boosted Trees (GBT) 
Operates by building decision trees iteratively. In each iteration, a new tree is fit to the residual errors of the previous tree to correct its mistakes. As trees are added, the model becomes more accurate, especially where the task is complex. GBT is known for its efficiency and effectiveness, often producing superior results with less data preprocessing compared to other algorithms.

## Result table + Confusion Matrix

The three classifiers we employed displayed competitive performances. The Logistic Regression achieved an AUC of 0.7735 in just 40 minutes, making it the quickest model. It was optimized with a regularization parameter (regParam) of 1e-05 and an elasticNetParam of 0.0. The Random Forest model posted an AUC of 0.762, requiring 3.03 hours, and was best configured with 20 trees and a maximum depth of 10. The Gradient-Boosted Trees (GBT) had the highest AUC of 0.788, and took the second to train longest at 3.01 hours. Its optimal parameters were a maximum iteration (maxIter) of 25, a maximum depth of 10, and a step size of 0.05. Notably, halving the learning rate for GBT and increasing maxIter prolonged its convergence.

`Training Size`   - 23669325 (2015-2018)

`Validation Size` - 7154007 (2019)

From the best-performing model, our confusion matrix revealed the following rates:

###### True Positives: 38.66%
###### True Negatives: 95.79%
###### False Negatives: 61.34%
###### False Positives: 4.21%.

|       | Logistic Regression | Random Forests | Gradient-Boosted Trees |
|-------|:-------------------:|:--------------:|:----------------------:|
| **AUC** |       .7735        |      .762       |        .788             |
| **Time**  |     40 minutes       |  3.03 Hours    |     3.01 hours          |

## Confusion Matrix of Best Classifier, GBT:
![BEST_AUC_GBT_7884](/files/tables/BEST_AUC_GBT_7884.png)

## Deep Learning Discussion - What went wrong?

In the course of our project, we ventured into implementing deep learning models, specifically an LSTM (Long Short-Term Memory) and a multi-layer perceptron. These architectures are often hailed for their prowess in handling sequential data and high-dimensional feature spaces, respectively. However, our experience was not seamless.

The LSTM, tailored for time series and sequence prediction tasks, struggled considerably. The achieved accuracy was merely 20%, which is markedly lower than what's expected from such sophisticated models. Similarly, the multi-layer perceptron, a kind of feed-forward artificial neural network, couldn't surpass an AUC of 0.50, rendering it no better than random guessing.

Our primary challenge arose from the distributed nature of Databricks. While distributed computing systems, like Spark on Databricks, excel in parallel data processing tasks, deep learning models, particularly LSTMs, require sequential data processing and often benefit from tight, low-latency inter-process communication. These inherent characteristics often clash with the distributed ethos. Additionally, deep learning models are computationally intensive and thrive with GPU acceleration. Implementing these models in Databricks without dedicated GPU resources put us at a disadvantage.

However, it's worth noting that with the right infrastructure – particularly, GPU support – the story might have been different. GPUs have been the linchpin in the rise of deep learning due to their parallel processing capabilities, making operations like matrix multiplications – a backbone of neural network computations – significantly faster.

In hindsight, while Databricks offers many advantages for big data and distributed ML tasks, when it comes to specialized deep learning architectures like LSTM, a dedicated deep learning environment equipped with GPU acceleration would likely have been more conducive.

## Gap Analysis

Our model is built strongly under the knowledge of previous plane route delays, knowledge of weather conditions, and specific details about the flight (airline, flight path, among others), but these are not all-encompassing.

There are bound to be flights with previous delays and bad weather that depart on time, frontier airlines flights that actually leave on time, or flights with perfect weather and no previous delays that are delayed for another reason. Based on the implementation of our ensemble methods we will miss these.

Especially with a model that derives so much predictiveness from one feature (a previous delay for the same plane used for the upcoming flight), we would expect the feature to cause mispredictions when it is not the right heuristic.

Compared to other teams, our AUC was more-or-less optimal, with one team achieving .795 AUC on the leaderboard.